# Aula 1 - Tratando os dados

In [ ]:
import pandas as pd

In [ ]:
cadastrados = pd.read_csv("dados/clientes_cadastrados.csv", sep=",")
cadastrados.head(3)

In [ ]:
cadastrados.shape

In [ ]:
cadastrados["ID_Cliente"].value_counts() #existem ids duplicados

In [ ]:
cadastrados[cadastrados["ID_Cliente"].duplicated(keep=False)].shape
# existem 94 ids duplicados

In [ ]:
ids_repetidos = cadastrados[cadastrados["ID_Cliente"].duplicated(keep=False)]

In [ ]:
cadastrados_limpo = cadastrados.drop(ids_repetidos.index)

In [ ]:
cadastrados_limpo["ID_Cliente"].value_counts()

In [ ]:
# tratando dados nulos
cadastrados_limpo.isnull().sum()

In [ ]:
cadastrados_limpo["Ocupacao"].unique()

In [ ]:
cadastrados_limpo["Ocupacao"].fillna(value="Outro", inplace=True)

In [ ]:
cadastrados_limpo.isnull().sum()

In [ ]:
cadastrados_limpo.nunique()

In [ ]:
cadastrados_limpo.drop("Genero", axis=1, inplace=True)

In [ ]:
cadastrados_limpo.drop("Tem_celular", axis=1, inplace=True)

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo["Tem_carro"].unique()

In [ ]:
cadastrados_limpo["Tem_casa_propria"].unique()

In [ ]:
cadastrados_limpo["Tem_telefone_trabalho"].unique()

In [ ]:
cadastrados_limpo["Tem_telefone_fixo"].unique()

In [ ]:
cadastrados_limpo.info()

In [ ]:
cadastrados_limpo["Tem_carro"] = cadastrados_limpo["Tem_carro"].replace(["Y", "N"], [1,0])

In [ ]:
cadastrados_limpo["Tem_casa_propria"] = cadastrados_limpo["Tem_carro"].replace(["Y", "N"], [1,0])

In [ ]:
cadastrados_limpo["Tem_casa_propria"].unique()

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo["Idade"] = - cadastrados_limpo["Idade"]/365.2425

In [ ]:
cadastrados_limpo["Anos_empregado"] = - cadastrados_limpo["Anos_empregado"]/365.2425

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo["Anos_empregado"].value_counts()

In [ ]:
cadastrados_limpo.loc[cadastrados_limpo['Anos_empregado']<0, 'Anos_empregado'] = 0

In [ ]:
cadastrados_limpo["Anos_empregado"].value_counts()

In [ ]:
cadastrados_limpo_bin = cadastrados_limpo

In [ ]:
cadastrados_limpo_bin.head(3)

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo_bin = cadastrados_limpo_bin.drop(['Categoria_de_renda', 'Grau_escolaridade', 'Estado_civil', 'Moradia', 'Ocupacao'], axis=1)

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo_bin.head(3)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
corr = cadastrados_limpo_bin.corr()
plt.figure(figsize = (10,10))
sns.heatmap(corr, cmap="Blues", annot=True)

In [ ]:
cadastrados_limpo_bin.drop(['Qtd_filhos'], axis=1, inplace=True)

In [ ]:
cadastrados_limpo = cadastrados_limpo[['ID_Cliente', 'Tem_carro', 'Tem_casa_propria', 
                                        'Tem_telefone_trabalho', 'Tem_telefone_fixo', 'Tem_email',
                                        'Tamanho_familia', 'Rendimento_anual', 'Idade', 'Anos_empregado',
                                        'Categoria_de_renda', 'Grau_escolaridade', 'Estado_civil',
                                        'Moradia', 'Ocupacao']]

In [ ]:
cadastrados_limpo.head(3)

In [ ]:
cadastrados_limpo['Categoria_de_renda'].value_counts(normalize=True)*100

# Aula 2 - Criando a Variável Target

In [ ]:
aprovados = pd.read_csv("dados/clientes_aprovados.csv")
aprovados.head()

In [ ]:
aprovados.info()

In [ ]:
aprovados.query("ID_Cliente == 5001712")

In [ ]:
aprovados.query("ID_Cliente == 5001711")

In [ ]:
aprovados.Faixa_atraso.value_counts()

In [ ]:
df_registros_creditos_agrupados_ID = aprovados.groupby('ID_Cliente')
ser_abertura = df_registros_creditos_agrupados_ID.apply(lambda x: min(x["Mes_referencia"]))
ser_abertura.name = "Abertura"
ser_abertura

In [ ]:
df_clientes_aprovados = aprovados.merge(ser_abertura, on="ID_Cliente")

In [ ]:
df_clientes_aprovados.head()

In [ ]:
ser_final = df_registros_creditos_agrupados_ID.apply(lambda x: max(x["Mes_referencia"]))
ser_final.name = "Final"
ser_final

In [ ]:
df_clientes_aprovados = df_clientes_aprovados.merge(ser_final, on="ID_Cliente")
df_clientes_aprovados.head()

In [ ]:
df_clientes_aprovados["Janela"] = df_clientes_aprovados['Final'] - df_clientes_aprovados['Abertura']
df_clientes_aprovados.head()

In [ ]:
df_clientes_aprovados["MOB"] = df_clientes_aprovados["Mes_referencia"] - df_clientes_aprovados["Abertura"]
df_clientes_aprovados.head()

In [ ]:
# transformando a faixa de atraso em faixas numéricas
df_clientes_aprovados["Faixa_atraso"].unique()

In [ ]:
dict_faixa_atraso_ind = {'nenhum empréstimo' :0, 
                         'pagamento realizado' :1, 
                         '1-29 dias' :2,
                         '30-59 dias' :3, 
                         '60-89 dias' :4, 
                         '90-119 dias' :5, 
                         '120-149 dias' :6,
                         '>150 dias' :7}

In [ ]:
df_clientes_aprovados['Ind_faixa_atraso'] = df_clientes_aprovados["Faixa_atraso"].map(dict_faixa_atraso_ind)
df_clientes_aprovados.head()

In [ ]:
df_clientes_aprovados["Mau_pagador"] = df_clientes_aprovados.apply(lambda x: 1 if x["Ind_faixa_atraso"] >3 else 0, axis=1)
df_clientes_aprovados.head()

In [ ]:
df_clientes_aprovados["Mau_pagador"].value_counts()

In [ ]:
df_registros_creditos_ID = df_clientes_aprovados[["ID_Cliente", 
                                                  "Abertura", 
                                                  "Final", 
                                                  "Janela"]].groupby("ID_Cliente").apply(lambda x: x.iloc[0]).reset_index(drop=True)

In [ ]:
df_registros_creditos_ID.head()

In [ ]:
df_denominador = df_registros_creditos_ID.groupby("Abertura").apply(lambda x: x["ID_Cliente"].count()).reset_index()
df_denominador.columns = ["Abertura", "Qtd_clientes"]
df_denominador

In [ ]:
df_vintage = df_clientes_aprovados.groupby(['Abertura', 'MOB']).apply(lambda x: x["ID_Cliente"].count()).reset_index()
df_vintage.columns = ["Abertura", "MOB", "Qtd_clientes"]
df_vintage

In [ ]:
df_vintage = pd.merge(df_vintage[["Abertura", "MOB"]], df_denominador,
                      on="Abertura", how="left")

In [ ]:
df_vintage

In [ ]:
import numpy as np

In [ ]:
df_vintage["Qtd_mau"] = np.nan

for j in range(-60, 1):
    ls = []
    for i in range(0, 61):
        due = list(df_clientes_aprovados.query('Mau_pagador == 1 and MOB ==@i and Abertura ==@j')["ID_Cliente"])
        ls.extend(due)
        df_vintage.loc[(df_vintage['MOB']== i) & (df_vintage['Abertura']==j), 'Qtd_mau'] = len(set(ls))

df_vintage["Taxa_de_mau"] = df_vintage['Qtd_mau'] / df_vintage['Qtd_clientes']
df_vintage

In [ ]:
df_vintage_pivot = df_vintage.pivot(index= "Abertura",
                                    columns='MOB',
                                    values="Taxa_de_mau")

In [ ]:
df_vintage_pivot

In [ ]:
lst = [i for i in range(0,61)]
df_vintage_pivot[lst].T.iloc[:, :10].plot(legend=True, grid=True, 
                                          title='Porcentagem Acumulada de Maus Pagadores - Atraso > 60 dias')
plt.xlabel('MOB')
plt.ylabel('Porcentagem acumulada de Maus Pagadores')
plt.legend(bbox_to_anchor=(1.2, 1), title='Abertura')
plt.show()

In [ ]:
sns.histplot(data=df_registros_creditos_ID, x='Janela', cumulative=True, 
             stat='density', kde=True)
plt.title('Distribuição de clientes por janela')
plt.xlabel('Janela')
plt.ylabel('Percentual de Clientes')

In [ ]:
dict_intervalo_atraso = {'maior_30_dias': 3,
                         'maior_60_dias': 4,
                         'maior_90_dias': 5,
                         'maior_120_dias': 6,
                         'maior_150_dias': 7}

In [ ]:
for chave, valor in dict_intervalo_atraso.items():
    df_clientes_aprovados[f'Mau_{chave}'] = df_clientes_aprovados.apply(lambda x:1 if x['Ind_faixa_atraso'] >= valor else 0, axis=1)

df_clientes_aprovados.head(3)

In [ ]:
dict_taxa_mau = {}
id_sum = len(set(df_clientes_aprovados['ID_Cliente']))

for chave in dict_intervalo_atraso.keys():
  df_min_mau = df_clientes_aprovados.query(f'Mau_{chave} == 1').groupby('ID_Cliente')['MOB'].min().reset_index()
  df_mob_taxa_mau = pd.DataFrame({'MOB':range(0,61), 'Taxa_Mau': np.nan})
  lst = []
  for i in range(0,61):
      due = df_min_mau.query('MOB == @i')['ID_Cliente'].to_list()
      lst.extend(due) #cumsum
      df_mob_taxa_mau.loc[df_mob_taxa_mau['MOB'] == i, 'Taxa_Mau'] = len(set(lst)) / id_sum
  dict_taxa_mau[chave] = df_mob_taxa_mau['Taxa_Mau']

df_taxa_mau = pd.DataFrame(dict_taxa_mau)
df_taxa_mau

In [ ]:
df_taxa_mau.plot(grid = True, title = '% acumulado de clientes maus para diversas faixas de atraso', figsize=(10, 6))
plt.xlabel('MOB')
plt.ylabel('% acumulado de clientes')
plt.show()

In [ ]:
df_clientes_aprovados_tratamento1 = df_clientes_aprovados.query('Janela >= 12')

In [ ]:
df_clientes_aprovados_tratamento1['Faixa_atraso'].unique()

In [ ]:
def verifica(registros):
    lista_status = registros['Faixa_atraso'].tolist()
    if '60-89 dias' in lista_status or '90-119 dias' in lista_status or '120-149 dias' in lista_status or '>150 dias' in lista_status:
        return 1
    else:
        return 0 
    
df_registros_creditos_id = pd.DataFrame(df_clientes_aprovados_tratamento1.groupby('ID_Cliente').apply(verifica)).reset_index()

df_registros_creditos_id.columns = ['ID_Cliente', 'Mau']

In [ ]:
df_registros_creditos_id.head()

In [ ]:
df_registros_clientes_target = cadastrados_limpo.merge(df_registros_creditos_id, on='ID_Cliente')

In [ ]:
df_registros_clientes_target.head(3)

In [ ]:
df_registros_clientes_target['Mau'].value_counts(normalize=True)*100

# Aula 3 - Construindo a Pipeline

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
seed = 1561651

In [ ]:
df_treino, df_teste = train_test_split(df_registros_clientes_target, test_size=0.2, random_state=seed)

In [ ]:
df_treino.shape

In [ ]:
df_teste.shape

In [ ]:
df_treino_copy = df_treino.copy()
df_teste_copy = df_teste.copy()

In [ ]:
#salvando os arquivos em CSV
df_treino.to_csv('train.csv', index=False)
df_teste.to_csv('teste.csv', index=False)

In [ ]:
df_treino.head(3)

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTE

In [ ]:
class class_drop_features(BaseEstimator, TransformerMixin):
    def __init__(self, feature_to_drop = ['ID_Cliente']):
        self.feature_to_drop = feature_to_drop
    def fit(self, df):
        return self
    def transform(self, df):
        if(set(self.feature_to_drop).issubset(df.columns)):
            df.drop(self.feature_to_drop, axis=1, inplace=True)
            return df
        else:
            print('Uma ou mais features não estão no Dataframe - drop_features')
            return df

In [ ]:
class class_min_max(BaseEstimator, TransformerMixin):
    def __init__(self, min_max_scaler = ['Tamanho_familia', 'Rendimento_anual', 'Idade', 'Anos_empregado']):
        self.min_max_scaler = min_max_scaler
    def fit(self, df):
        return self
    def transform(self, df):
        if (set(self.min_max_scaler).issubset(df.columns)):
            min_max_enc = MinMaxScaler()
            df[self.min_max_scaler] = min_max_enc.fit_transform(df[self.min_max_scaler])
            return df
        else:
            print('Uma ou mais features não estão no Dataframe - min_max')
            return df

In [ ]:
class class_one_hot_enc_names(BaseEstimator, TransformerMixin):
    def __init__(self, one_hot_encoding = ['Categoria_de_renda', 'Estado_civil', 'Moradia', 'Ocupacao']):
        self.one_hot_encoding = one_hot_encoding
    
    def fit(self, df):
        return self
    
    def transform(self, df):
        if(set(self.one_hot_encoding).issubset(df.columns)):
            # função para rodar o OneHotEncoder nas features
            def onehot_features(df, one_hot_encoding):
                onehot_features = OneHotEncoder()
                onehot_features.fit(df[one_hot_encoding])
                
                #obtendo o nome das colunas
                feature_names = onehot_features.get_feature_names_out(one_hot_encoding)
                
                # transformando o array em um dataframe com o nome das colunas
                df = pd.DataFrame(onehot_features.transform(df[self.one_hot_encoding]).toarray(),
                                  columns= feature_names, index=df.index)
                
                return df
            
            #função para concatenar as features que passaram pelo fit dos dados com o restante 
            def concat_df(df, onehotenc_df, one_hot_encoding):
                outras_features = [feature for feature in df.columns if feature not in one_hot_encoding]
                df_concat = pd.concat([onehotenc_df, df[outras_features]], axis=1)
                return df_concat
            
            df_onehotencoding = onehot_features(df, self.one_hot_encoding)
            
            # retorna o df concatenado
            df_full = concat_df(df, df_onehotencoding, self.one_hot_encoding)
            return df_full
        
        else:
            print('Uma ou mais features não estão no Dataframe - min_max')
            return df

In [ ]:
class class_ordinal_feature(BaseEstimator, TransformerMixin):
    def __init__(self, ordinal_feature = ['Grau_escolaridade']):
        self.ordinal_feature = ordinal_feature
    
    def fit(self, df):
        return self
    
    def transform(self, df):
        if 'Grau_escolaridade' in df.columns:
            ord_encoder = OrdinalEncoder()
            df[self.ordinal_feature] = ord_encoder.fit_transform(df[self.ordinal_feature])
            return df
        else:
            print('Grau_escolaridade não está no Dataframe')
            return df

In [ ]:
class class_oversample(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, df):
        return self
    
    def transform(self, df):
        if 'Mau' in df.columns:
            # usando a função SMOTE para superamostrar a clase minoritária e corrigir dados desbalanceados
            oversample = SMOTE(sampling_strategy='minority')
            X_bal, y_bal = oversample.fit_resample(df.loc[:, df.columns !='Mau'], df['Mau'])
            df_balanceado = pd.concat([pd.DataFrame(X_bal), pd.DataFrame(y_bal)], axis=1)
            return df_balanceado
        else:
            print("O target não está no Dataframe")
            return df

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
def df_pipeline(df):
    pipeline = Pipeline([
        ('feature_dropper', class_drop_features()),
        ('MinMaxScaler', class_min_max()),
        ('OneHotEncoding', class_one_hot_enc_names()),
        ('OrdinalFeatures', class_ordinal_feature()),
        ('Oversample', class_oversample())
    ])
    
    df_pipeline = pipeline.fit_transform(df)
    return df_pipeline

# Aula 4 - Machine Learning: Escolhendo o melhor modelo

In [ ]:
treino = df_pipeline(df_treino)

In [ ]:
treino.head()

In [ ]:
treino['Mau'].value_counts(normalize=True)*100 # agora os dados não estão mais desbalanceados

In [ ]:
treino.shape

In [ ]:
X_treino, y_treino = treino.loc[:, treino.columns != 'Mau'], treino['Mau'] # y é sempre a nossa coluna target

In [ ]:
teste = df_pipeline(df_teste)

In [ ]:
teste.head()

In [ ]:
X_teste, y_teste = teste.loc[:, teste.columns != 'Mau'], teste['Mau'] 

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, roc_auc_score, RocCurveDisplay
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip show scikit_learn

In [ ]:
def roda_modelo(modelo):
    modelo.fit(X_treino, y_treino)
    
    prob_predict = modelo.predict_proba(X_teste)
    
    print(f'\n---------------Resultados{modelo}---------------\n')
    
    data_bom = np.sort(modelo.predict_proba(X_teste)[:, 0])
    data_mau = np.sort(modelo.predict_proba(X_teste)[:, 1])
    kstest = stats.ks_2samp(data_bom, data_mau)
    
    print(f'Métrica KS: {kstest}')
    
    print(f'\nConfusion Matrix\n')
    
    fig, ax = plt.subplots(figsize=(7,7))
    matriz_confusao = ConfusionMatrixDisplay(modelo, X_teste, y_teste, normalize='true',
                                            display_labels=['Bom pagador', 'Mau pagador'],
                                            ax=ax, cmap=plt.cm.Blues)
    ax.set_title('Matriz de Confusão\nNormalizada', fontsize=16, fontweight='bold')
    ax.set_xlabel('Label predita', fontsize=18)
    ax.set_ylabel('Label verdadeira', fontsize=18)
    plt.grid(False)
    plt.show(matriz_confusao)
    
    predicao = modelo.predict(X_teste)
    print(f'\nClassification Report\n')
    print(classification_report(y_teste, predicao, zero_division=0))
    
    print(f'\nROC curve\n')
    metrics.RocCurveDisplay(modelo, X_teste, y_teste)

In [ ]:
# importando os modelos de Machine Learning 
from sklearn.linear_model import LogisticRegression
modelo_logistico = LogisticRegression()

In [ ]:
roda_modelo(modelo_logistico)